## Exploring the banking dataset using AutoLabel

#### Setup the API Keys for providers that you want to use

In [1]:
# %env OPENAI_API_KEY=***REMOVED***
%env OPENAI_API_KEY=***REMOVED***
%env ANTHROPIC_API_KEY=***REMOVED***

env: OPENAI_API_KEY=***REMOVED***
env: ANTHROPIC_API_KEY=***REMOVED***


### Get the correct data

First lets make sure that we download the correct data. Currently data has been hosted on s3 but we will upload it to huggingface data as well in the future.

In [12]:
!aws s3 cp s3://refuel-benchmarking data/ --recursive

download: s3://refuel-benchmarking/banking_seed.csv to data/banking_seed.csv
download: s3://refuel-benchmarking/conll2003_seed.csv to data/conll2003_seed.csv
download: s3://refuel-benchmarking/emotion_seed.csv to data/emotion_seed.csv
download: s3://refuel-benchmarking/banking_test.csv to data/banking_test.csv
download: s3://refuel-benchmarking/civil_comments_seed.csv to data/civil_comments_seed.csv
download: s3://refuel-benchmarking/company_seed.csv to data/company_seed.csv
download: s3://refuel-benchmarking/emotion_test.csv to data/emotion_test.csv
download: s3://refuel-benchmarking/ledgar_seed.csv to data/ledgar_seed.csv
download: s3://refuel-benchmarking/conll2003_test.csv to data/conll2003_test.csv
download: s3://refuel-benchmarking/civil_comments_test.csv to data/civil_comments_test.csv
download: s3://refuel-benchmarking/medqa_seed.csv to data/medqa_seed.csv
download: s3://refuel-benchmarking/pubmed_qa_seed.csv to data/pubmed_qa_seed.csv
download: s3://refuel-benchmarking/sciq_se

### Run the labeler after passing in my own seed examples

#### Create the config
This config file has all the possible labels for the banking dataset. The model needs to choose one label from the label list provided. In input schema, we define the input columns that will be used by the oracle and the output column defines the label column, that is, the column that will be used as ground truth and will be tried to be generated by our library

In [4]:
zero_shot_config = {
    "task_name": "BankingComplaintsClassification",
    "task_type": "classification",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo"
    },
    "prompt": {
        "task_guidelines": "You are an expert at understanding bank customers support complaints and queries.\nYour job is to correctly classify the provided input example into one of the following categories.\nCategories:\n{labels}",
        "output_guidelines": "You will answer with just the the correct output label and nothing else.",
        "labels": [
            "activate_my_card",
            "age_limit",
            "apple_pay_or_google_pay",
            "atm_support",
            "automatic_top_up",
            "balance_not_updated_after_bank_transfer",
            "balance_not_updated_after_cheque_or_cash_deposit",
            "beneficiary_not_allowed",
            "cancel_transfer",
            "card_about_to_expire",
            "card_acceptance",
            "card_arrival",
            "card_delivery_estimate",
            "card_linking",
            "card_not_working",
            "card_payment_fee_charged",
            "card_payment_not_recognised",
            "card_payment_wrong_exchange_rate",
            "card_swallowed",
            "cash_withdrawal_charge",
            "cash_withdrawal_not_recognised",
            "change_pin",
            "compromised_card",
            "contactless_not_working",
            "country_support",
            "declined_card_payment",
            "declined_cash_withdrawal",
            "declined_transfer",
            "direct_debit_payment_not_recognised",
            "disposable_card_limits",
            "edit_personal_details",
            "exchange_charge",
            "exchange_rate",
            "exchange_via_app",
            "extra_charge_on_statement",
            "failed_transfer",
            "fiat_currency_support",
            "get_disposable_virtual_card",
            "get_physical_card",
            "getting_spare_card",
            "getting_virtual_card",
            "lost_or_stolen_card",
            "lost_or_stolen_phone",
            "order_physical_card",
            "passcode_forgotten",
            "pending_card_payment",
            "pending_cash_withdrawal",
            "pending_top_up",
            "pending_transfer",
            "pin_blocked",
            "receiving_money",
            "Refund_not_showing_up",
            "request_refund",
            "reverted_card_payment?",
            "supported_cards_and_currencies",
            "terminate_account",
            "top_up_by_bank_transfer_charge",
            "top_up_by_card_charge",
            "top_up_by_cash_or_cheque",
            "top_up_failed",
            "top_up_limits",
            "top_up_reverted",
            "topping_up_by_card",
            "transaction_charged_twice",
            "transfer_fee_charged",
            "transfer_into_account",
            "transfer_not_received_by_recipient",
            "transfer_timing",
            "unable_to_verify_identity",
            "verify_my_identity",
            "verify_source_of_funds",
            "verify_top_up",
            "virtual_card_not_working",
            "visa_or_mastercard",
            "why_verify_identity",
            "wrong_amount_of_cash_received",
            "wrong_exchange_rate_for_cash_withdrawal"
        ],
        "example_template": "Input: {example}\nOutput: {label}"
    }
}


## Run the model

First dry run the model using the above specification and get an idea of the cost required to run the model 

Running it on just 100 examples to get an idea for the notebook, adjust the max_items as required and dont pass it if you need to run the oracle on the whole dataset.

In [2]:
from autolabel import LabelingAgent

In [6]:
agent = LabelingAgent(config=zero_shot_config)

In [7]:

agent.plan('../data/banking_test.csv', max_items=10)


Output()

Total Estimated Cost: $0.029
Number of examples to label: 10
Average cost per example: $0.00292


A prompt example:

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_

Now, actually run the model and generate the list of labels for the banking dataset. You will get the computed metrics at the end of the run.

In [8]:
labels, df, metrics_list = agent.run('../data/banking_test.csv', max_items = 50)

Output()

2023-05-30 17:09:10 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 25b4db0b913e95ba2361d8d599c9b3d6 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:09:10 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 25b4db0b913e95ba2361d8d599c9b3d6 in your message.).


Metric: Actual Cost: 0.04632399999999999
Metric: support: [(50, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.68, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


## Change the prompt
Analyze the dataframe generated above and make changes to the prompt in order to get better results

In [10]:
config_fixed_few_shot = {
    "task_name": "BankingComplaintsClassification",
    "task_type": "classification",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo"
    },
    "prompt": {
        "task_guidelines": "You are an expert at understanding bank customers support complaints and queries.\nYour job is to correctly classify the provided input example into one of the following categories.\nCategories:\n{labels}",
        "output_guidelines": "You will answer with just the the correct output label and nothing else.",
        "labels": [
            "activate_my_card",
            "age_limit",
            "apple_pay_or_google_pay",
            "atm_support",
            "automatic_top_up",
            "balance_not_updated_after_bank_transfer",
            "balance_not_updated_after_cheque_or_cash_deposit",
            "beneficiary_not_allowed",
            "cancel_transfer",
            "card_about_to_expire",
            "card_acceptance",
            "card_arrival",
            "card_delivery_estimate",
            "card_linking",
            "card_not_working",
            "card_payment_fee_charged",
            "card_payment_not_recognised",
            "card_payment_wrong_exchange_rate",
            "card_swallowed",
            "cash_withdrawal_charge",
            "cash_withdrawal_not_recognised",
            "change_pin",
            "compromised_card",
            "contactless_not_working",
            "country_support",
            "declined_card_payment",
            "declined_cash_withdrawal",
            "declined_transfer",
            "direct_debit_payment_not_recognised",
            "disposable_card_limits",
            "edit_personal_details",
            "exchange_charge",
            "exchange_rate",
            "exchange_via_app",
            "extra_charge_on_statement",
            "failed_transfer",
            "fiat_currency_support",
            "get_disposable_virtual_card",
            "get_physical_card",
            "getting_spare_card",
            "getting_virtual_card",
            "lost_or_stolen_card",
            "lost_or_stolen_phone",
            "order_physical_card",
            "passcode_forgotten",
            "pending_card_payment",
            "pending_cash_withdrawal",
            "pending_top_up",
            "pending_transfer",
            "pin_blocked",
            "receiving_money",
            "Refund_not_showing_up",
            "request_refund",
            "reverted_card_payment?",
            "supported_cards_and_currencies",
            "terminate_account",
            "top_up_by_bank_transfer_charge",
            "top_up_by_card_charge",
            "top_up_by_cash_or_cheque",
            "top_up_failed",
            "top_up_limits",
            "top_up_reverted",
            "topping_up_by_card",
            "transaction_charged_twice",
            "transfer_fee_charged",
            "transfer_into_account",
            "transfer_not_received_by_recipient",
            "transfer_timing",
            "unable_to_verify_identity",
            "verify_my_identity",
            "verify_source_of_funds",
            "verify_top_up",
            "virtual_card_not_working",
            "visa_or_mastercard",
            "why_verify_identity",
            "wrong_amount_of_cash_received",
            "wrong_exchange_rate_for_cash_withdrawal"
        ],
        "few_shot_examples": [
            {
                "example": "Is it free to transfer money or is there a fee?",
                "label": "transfer_fee_charged"
            },
            {
                "example": "I need my PIN, where is it?",
                "label": "get_physical_card"
            },
            {
                "example": "Can my salary be received and transferred to my current currency in my country?",
                "label": "receiving_money"
            },
            {
                "example": "Why isn't my purchase exchange rate correct?",
                "label": "card_payment_wrong_exchange_rate"
            },
            {
                "example": "Why was I charged a fee on a cash withdrawal?",
                "label": "cash_withdrawal_charge"
            }
        ],
        "few_shot_selection": "fixed",
        "few_shot_num": 5,
        "example_template": "Input: {example}\nOutput: {label}"
    }
}

In [11]:
agent = LabelingAgent(config_fixed_few_shot)
agent.plan('../data/banking_test.csv', max_items = 500)

Output()

Total Estimated Cost: $1.569
Number of examples to label: 500
Average cost per example: $0.00314


A prompt example:

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange

In [12]:
labels, df, metrics_list = agent.run('../data/banking_test.csv', max_items=500)

Output()

2023-05-30 17:13:55 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88375 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 17:13:55 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88375 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 17:14:00 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88442 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 17:14:00 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88442 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 17:14:04 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88472 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 17:14:04 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88472 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 17:14:06 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88448 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 17:14:06 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88448 / min. Contact us through our help center at help.openai.com if you continue to have issues..


...

2023-05-30 17:20:47 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89232 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 17:20:47 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89232 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 17:20:48 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88564 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 17:20:48 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88564 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 17:20:50 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88915 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 17:20:50 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 88915 / min. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 17:20:52 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89134 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 17:20:52 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89134 / min. Contact us through our help center at help.openai.com if you continue to have issues..


Metric: Actual Cost: 0.5308739999999998
Metric: support: [(495, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.7353535353535353, 'index=0')]
Metric: completion_rate: [(0.99, 'index=0')]
Total number of failures: 0


## Example selector
Let's try to use an example selector. Let's use the big seed example set so that we can choose from a bigger set of examples

In [5]:
config_similarity_few_shot = {
    "task_name": "BankingComplaintsClassification",
    "task_type": "classification",
    "dataset": {
        "label_column": "label",
        "delimiter": ","
    },
    "model": {
        "provider": "openai",
        "name": "gpt-3.5-turbo"
        # "params": {"max_tokens": 512},
    },
    "prompt": {
        "task_guidelines": "You are an expert at understanding bank customers support complaints and queries.\nYour job is to correctly classify the provided input example into one of the following categories.\nCategories:\n{labels}",
        "output_guidelines": "You will answer with just the the correct output label and nothing else.",
        "labels": [
            "activate_my_card",
            "age_limit",
            "apple_pay_or_google_pay",
            "atm_support",
            "automatic_top_up",
            "balance_not_updated_after_bank_transfer",
            "balance_not_updated_after_cheque_or_cash_deposit",
            "beneficiary_not_allowed",
            "cancel_transfer",
            "card_about_to_expire",
            "card_acceptance",
            "card_arrival",
            "card_delivery_estimate",
            "card_linking",
            "card_not_working",
            "card_payment_fee_charged",
            "card_payment_not_recognised",
            "card_payment_wrong_exchange_rate",
            "card_swallowed",
            "cash_withdrawal_charge",
            "cash_withdrawal_not_recognised",
            "change_pin",
            "compromised_card",
            "contactless_not_working",
            "country_support",
            "declined_card_payment",
            "declined_cash_withdrawal",
            "declined_transfer",
            "direct_debit_payment_not_recognised",
            "disposable_card_limits",
            "edit_personal_details",
            "exchange_charge",
            "exchange_rate",
            "exchange_via_app",
            "extra_charge_on_statement",
            "failed_transfer",
            "fiat_currency_support",
            "get_disposable_virtual_card",
            "get_physical_card",
            "getting_spare_card",
            "getting_virtual_card",
            "lost_or_stolen_card",
            "lost_or_stolen_phone",
            "order_physical_card",
            "passcode_forgotten",
            "pending_card_payment",
            "pending_cash_withdrawal",
            "pending_top_up",
            "pending_transfer",
            "pin_blocked",
            "receiving_money",
            "Refund_not_showing_up",
            "request_refund",
            "reverted_card_payment?",
            "supported_cards_and_currencies",
            "terminate_account",
            "top_up_by_bank_transfer_charge",
            "top_up_by_card_charge",
            "top_up_by_cash_or_cheque",
            "top_up_failed",
            "top_up_limits",
            "top_up_reverted",
            "topping_up_by_card",
            "transaction_charged_twice",
            "transfer_fee_charged",
            "transfer_into_account",
            "transfer_not_received_by_recipient",
            "transfer_timing",
            "unable_to_verify_identity",
            "verify_my_identity",
            "verify_source_of_funds",
            "verify_top_up",
            "virtual_card_not_working",
            "visa_or_mastercard",
            "why_verify_identity",
            "wrong_amount_of_cash_received",
            "wrong_exchange_rate_for_cash_withdrawal"
        ],
        "few_shot_examples": "../data/banking_seed.csv",
        "few_shot_selection": "semantic_similarity",
        "few_shot_num": 10,
        "example_template": "Input: {example}\nOutput: {label}"
    }
}


In [15]:
agent = LabelingAgent(config_similarity_few_shot)


In [16]:
agent.plan('../data/banking_test.csv', max_items=100)


Output()

Total Estimated Cost: $0.335
Number of examples to label: 100
Average cost per example: $0.00335


A prompt example:

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange

In [17]:
labels, df, metrics_list = agent.run('../data/banking_test.csv', max_items = 500)

Output()

2023-05-30 17:34:33 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e11925cc2fabd92886b2f3e409c2ad91 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:34:33 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID e11925cc2fabd92886b2f3e409c2ad91 in your message.).


2023-05-30 17:36:25 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 69fcbeffe64c68718042a602a2225641 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:36:25 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 69fcbeffe64c68718042a602a2225641 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or co

2023-05-30 17:37:01 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID bc8355e65dfbeeb79b720ce46a4a0198 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:37:01 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID bc8355e65dfbeeb79b720ce46a4a0198 in your message.).


2023-05-30 17:39:07 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1f646166cd3c8476da173f5011db9e31 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:39:07 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1f646166cd3c8476da173f5011db9e31 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can re

...

2023-05-30 17:39:15 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 661d488d8cc64ec8a9cc64cc6ace6243 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:39:15 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 661d488d8cc64ec8a9cc64cc6ace6243 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can re

2023-05-30 17:39:17 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a87ed5d1bdd0e72cf4284b8e58054b6e in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:39:17 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID a87ed5d1bdd0e72cf4284b8e58054b6e in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can re

2023-05-30 17:39:20 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID ccc86f7092340078045cd675035d9451 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:39:20 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID ccc86f7092340078045cd675035d9451 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can re

2023-05-30 17:39:21 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d91b408446237e8ef2640b27f87f7d48 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:39:21 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d91b408446237e8ef2640b27f87f7d48 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can re

Metric: Actual Cost: 0.42560799999999993
Metric: support: [(500, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.772, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


In [19]:
## Davinci

from copy import deepcopy

config_v4 = deepcopy(config_similarity_few_shot)

config_v4['model'] = {'provider': 'openai', 'name': 'text-davinci-003'}


In [20]:
agent = LabelingAgent(config_v4)

In [22]:
agent.plan('../data/banking_test.csv', max_items=500)


Output()

Total Estimated Cost: $19.277
Number of examples to label: 500
Average cost per example: $0.03855


A prompt example:

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchang

In [24]:
labels, df, metrics_list = agent.run('../data/banking_test.csv', max_items=1000)


Output()

2023-05-30 17:54:29 openai INFO: error_code=None error_message='Request failed due to server shutdown' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 17:54:29 langchain.llms.openai WARNING: Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: Request failed due to server shutdown {
  "error": {
    "message": "Request failed due to server shutdown",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'Request failed due to server shutdown', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Wed, 31 May 2023 00:54:29 GMT', 'Content-Type': 'application/json', 'Content-Length': '141', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-model': 'text-davinci-003', 'openai-organization': 'refuel', 'openai-processing-ms': '2664', 'openai-version': '2020-10-01', 'strict-transport-security

Metric: Actual Cost: 9.71642
Metric: support: [(1000, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.779, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


In [26]:

config_v5 = deepcopy(config_similarity_few_shot)

config_v5['model'] = {'provider': 'anthropic', 'name': 'claude-v1'}


In [27]:
agent = LabelingAgent(config_v5)

In [31]:
agent.plan('../data/banking_test.csv', max_items=10)

Output()

Total Estimated Cost: $0.425
Number of examples to label: 10
Average cost per example: $0.04248


A prompt example:

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_

In [34]:
labels, df, metrics_list = agent.run('../data/banking_test.csv', max_items=1000)


2023-05-30 18:13:46.837 | INFO     | autolabel.labeler:run:137 - Task run already exists.
There is an existing task with following details: id='3143625989' created_at=datetime.datetime(2023, 5, 30, 18, 7, 40, 919962) task_id='51d5fc05d9b07351245e0bd41f9e7434' dataset_id='7a7ef3f81d1078751194b3e986ff48bf' current_index=35 output_file='../data/banking_test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None
Evaluating the existing task...
Metric: support: [(40, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.725, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
40 examples have been labeled so far.
Last annotated example - Prompt: You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_trans

Output()

2023-05-30 18:33:42 openai INFO: error_code=None error_message='The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists.' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 18:33:42 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: The server is currently overloaded with other requests. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists..


2023-05-30 18:34:16 openai INFO: error_code=None error_message='The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d81d230f052743cc6bf95f0632c1b236 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-30 18:34:16 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d81d230f052743cc6bf95f0632c1b236 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or co

Metric: Actual Cost: 8.988903040000002
Metric: support: [(1000, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.808, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


In [36]:

config_v6 = deepcopy(config_similarity_few_shot)

config_v6['model'] = {'provider': 'openai', 'name': 'gpt-4'}


In [37]:
agent = LabelingAgent(config_v6)
agent.llm


In [38]:
agent.plan('../data/banking_test.csv', max_items=10)

Output()

Total Estimated Cost: $0.804
Number of examples to label: 10
Average cost per example: $0.0804


A prompt example:

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_c

In [39]:
agent.run('../data/banking_test.csv', max_items=500)

Output()

2023-05-30 20:14:48 openai INFO: error_code=502 error_message='Bad gateway.' error_param=None error_type=cf_bad_gateway message='OpenAI API error received' stream_error=False
2023-05-30 20:14:48 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: Bad gateway. {"error":{"code":502,"message":"Bad gateway.","param":null,"type":"cf_bad_gateway"}} 502 {'error': {'code': 502, 'message': 'Bad gateway.', 'param': None, 'type': 'cf_bad_gateway'}} {'Date': 'Wed, 31 May 2023 03:14:48 GMT', 'Content-Type': 'application/json', 'Content-Length': '84', 'Connection': 'keep-alive', 'X-Frame-Options': 'SAMEORIGIN', 'Referrer-Policy': 'same-origin', 'Cache-Control': 'private, max-age=0, no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Expires': 'Thu, 01 Jan 1970 00:00:01 GMT', 'Server': 'cloudflare', 'CF-RAY': '7cfc04dfae71faa6-SJC', 'alt-svc': 'h3=":443"; ma=864

2023-05-30 20:19:47 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 20:19:47 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 20:19:55 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 20:19:55 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 20:19:57 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 20:19:57 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues..


...

2023-05-30 20:34:25 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 20:34:25 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 20:34:28 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 20:34:28 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues..


2023-05-30 20:34:41 openai INFO: error_code=None error_message='Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-05-30 20:34:41 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-4 in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 40000 / min. Please try again in 1ms. Contact us through our help center at help.openai.com if you continue to have issues..


Metric: Actual Cost: 10.230660000000004
Metric: support: [(500, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.816, 'index=0')]
Metric: completion_rate: [(1.0, 'index=0')]
Total number of failures: 0


(0                        terminate_account
 1                   cash_withdrawal_charge
 2      direct_debit_payment_not_recognised
 3                 card_payment_fee_charged
 4                extra_charge_on_statement
                       ...                 
 495                            pin_blocked
 496               card_payment_fee_charged
 497                        transfer_timing
 498                  edit_personal_details
 499                pending_cash_withdrawal
 Name: BankingComplaintsClassification_llm_label, Length: 500, dtype: object,
                                                example  \
 0                           I want to close my account   
 1    It seems I was overcharged when I used an ATM ...   
 2    I have a direct debit transaction I have not s...   
 3      How much does it cost in fees to use your card?   
 4                          There is an extra $1 charge   
 ..                                                 ...   
 495                    A

In [6]:
from copy import deepcopy

config_refuel = deepcopy(config_similarity_few_shot)

config_refuel['model'] = {'provider': 'refuel', 'name': 'flan-t5-xxl'}


In [7]:
agent = LabelingAgent(config_refuel)
agent.llm


In [42]:
agent.plan('../data/banking_test.csv', max_items=10)

2023-05-30 21:00:00 langchain.embeddings.openai WARNING: Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


Output()

Total Estimated Cost: $0.0
Number of examples to label: 10
Average cost per example: $0.0


A prompt example:

You are an expert at understanding bank customers support complaints and queries.
Your job is to correctly classify the provided input example into one of the following categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_charge

In [43]:
labels, df, metrics_list = agent.run('../data/banking_test.csv', max_items=500)


Output()

2023-05-30 21:01:27.362 | WARNING  | tenacity.before_sleep:log_it:65 - Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 2.0 seconds as it raised HTTPError: 502 Server Error: Bad Gateway for url: https://llm.refuel.ai/.


2023-05-30 21:01:29.599 | WARNING  | tenacity.before_sleep:log_it:65 - Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 2.0 seconds as it raised HTTPError: 502 Server Error: Bad Gateway for url: https://llm.refuel.ai/.


2023-05-30 21:01:31.806 | WARNING  | tenacity.before_sleep:log_it:65 - Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 4.0 seconds as it raised HTTPError: 502 Server Error: Bad Gateway for url: https://llm.refuel.ai/.


2023-05-30 21:01:36.023 | WARNING  | tenacity.before_sleep:log_it:65 - Retrying autolabel.models.refuel.RefuelLLM._label_with_retry in 8.0 seconds as it raised HTTPError: 502 Server Error: Bad Gateway for url: https://llm.refuel.ai/.


KeyboardInterrupt: 

## Confidence estimation
Let's try to see if the model is able to do well on confidence estimation, if the model knows what it doesn't know we might be able to trade off completion rate for accuracy and get a higher accuracy even though we may be labeling less amount of data.

In [22]:
task_config = {
    "project_name": "BankingClassification",
    "task_type": "classification",
    "prefix_prompt": "You are an expert at understanding twitter complaints.",
    "example_selector": {
        "strategy": "semantic_similarity",
        "num_examples": 4
    },
    "compute_confidence": "True"
}

In [25]:
o = LabelingAgent(task_config, llm_config)
o.plan('data/banking_test.csv', dataset_config, max_items = 100)

100%|██████████| 20/20 [00:24<00:00,  1.23s/it]

Total Estimated Cost: $0.135
Number of examples to label: 100
Average cost per example: $0.00135


A prompt example:

You are an expert at understanding twitter complaints.
Your job is to correctly label the provided input example into one of the following 77 categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_charge
exchange_rate
excha

In [26]:
labels, df, metrics_list = o.run('data/banking_test.csv', dataset_config, max_items = 100)

2023-05-08 22:21:12.180 | INFO     | refuel_oracle.oracle:annotate:122 - Task run already exists.
There is an existing task with following details: id='4220919502' created_at=datetime.datetime(2023, 5, 8, 22, 12, 21, 695444) task_id='fe18616c502136eecf138fd6f0fbd7a9' dataset_id='8d146e04eeed04671abdece0f40e0469' current_index=65 output_file='data/banking_test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None
Evaluating the existing task...


/home/ubuntu/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Metric: auroc: 0.9310009718172984
Metric: support: [(70, 'index=0'), (0, 'index=1'), (1, 'index=2'), (19, 'index=3'), (20, 'index=4'), (39, 'index=5'), (40, 'index=6'), (41, 'index=7'), (43, 'index=8'), (50, 'index=9'), (52, 'index=10'), (53, 'index=11'), (55, 'index=12'), (57, 'index=13'), (70, 'index=14')]
Metric: threshold: [(-inf, 'index=0'), (3.7169041911318668, 'index=1'), (2.7169041911318668, 'index=2'), (2.7095740903214547, 'index=3'), (2.709052961047316, 'index=4'), (2.6906020486746898, 'index=5'), (2.6903324708649907, 'index=6'), (2.687156017308222, 'index=7'), (2.6787913093093048, 'index=8'), (2.630019572879165, 'index=9'), (2.5956249750404234, 'index=10'), (2.5799757948748283, 'index=11'), (2.568780087012188, 'index=12'), (2.5532547033659325, 'index=13'), (2.2109919215757605, 'index=14')]
Metric: accuracy: [(0.7, 'index=0'), (nan, 'index=1'), (1.0, 'index=2'), (1.0, 'index=3'), (0.95, 'index=4'), (0.9743589743589743, 'index=5'), (0.95, 'index=6'), (0.9512195121951219, 'inde

Do you want to resume it? (y/n) y


Resuming the task...


100%|██████████| 7/7 [00:48<00:00,  6.95s/it]


Metric: auroc: 0.9205333333333333
Metric: support: [(100, 'index=0'), (0, 'index=1'), (1, 'index=2'), (29, 'index=3'), (30, 'index=4'), (35, 'index=5'), (36, 'index=6'), (61, 'index=7'), (62, 'index=8'), (64, 'index=9'), (66, 'index=10'), (76, 'index=11'), (79, 'index=12'), (80, 'index=13'), (82, 'index=14'), (85, 'index=15'), (100, 'index=16')]
Metric: threshold: [(-inf, 'index=0'), (3.716918815608781, 'index=1'), (2.716918815608781, 'index=2'), (2.7095342984271964, 'index=3'), (2.709052961047316, 'index=4'), (2.7074403430995178, 'index=5'), (2.70737698174731, 'index=6'), (2.6906020486746898, 'index=7'), (2.6903324708649907, 'index=8'), (2.6864060122848614, 'index=9'), (2.6787913093093048, 'index=10'), (2.6220833554670904, 'index=11'), (2.5956249750404234, 'index=12'), (2.5799757948748283, 'index=13'), (2.568780087012188, 'index=14'), (2.5532547033659325, 'index=15'), (2.2109919215757605, 'index=16')]
Metric: accuracy: [(0.75, 'index=0'), (nan, 'index=1'), (1.0, 'index=2'), (1.0, 'ind

/home/ubuntu/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


## Chain of thought reasoning

Chain of thought requires an explanation for every seed example. As we don't have the explanations or the domain knowledge to construct these explanations, we can use the model to generate the explanations and then use these explanations as an input to the model

In [32]:
dataset_config = {
    "labels_list": [
        "activate_my_card",
        "age_limit",
        "apple_pay_or_google_pay",
        "atm_support",
        "automatic_top_up",
        "balance_not_updated_after_bank_transfer",
        "balance_not_updated_after_cheque_or_cash_deposit",
        "beneficiary_not_allowed",
        "cancel_transfer",
        "card_about_to_expire",
        "card_acceptance",
        "card_arrival",
        "card_delivery_estimate",
        "card_linking",
        "card_not_working",
        "card_payment_fee_charged",
        "card_payment_not_recognised",
        "card_payment_wrong_exchange_rate",
        "card_swallowed",
        "cash_withdrawal_charge",
        "cash_withdrawal_not_recognised",
        "change_pin",
        "compromised_card",
        "contactless_not_working",
        "country_support",
        "declined_card_payment",
        "declined_cash_withdrawal",
        "declined_transfer",
        "direct_debit_payment_not_recognised",
        "disposable_card_limits",
        "edit_personal_details",
        "exchange_charge",
        "exchange_rate",
        "exchange_via_app",
        "extra_charge_on_statement",
        "failed_transfer",
        "fiat_currency_support",
        "get_disposable_virtual_card",
        "get_physical_card",
        "getting_spare_card",
        "getting_virtual_card",
        "lost_or_stolen_card",
        "lost_or_stolen_phone",
        "order_physical_card",
        "passcode_forgotten",
        "pending_card_payment",
        "pending_cash_withdrawal",
        "pending_top_up",
        "pending_transfer",
        "pin_blocked",
        "receiving_money",
        "Refund_not_showing_up",
        "request_refund",
        "reverted_card_payment?",
        "supported_cards_and_currencies",
        "terminate_account",
        "top_up_by_bank_transfer_charge",
        "top_up_by_card_charge",
        "top_up_by_cash_or_cheque",
        "top_up_failed",
        "top_up_limits",
        "top_up_reverted",
        "topping_up_by_card",
        "transaction_charged_twice",
        "transfer_fee_charged",
        "transfer_into_account",
        "transfer_not_received_by_recipient",
        "transfer_timing",
        "unable_to_verify_identity",
        "verify_my_identity",
        "verify_source_of_funds",
        "verify_top_up",
        "virtual_card_not_working",
        "visa_or_mastercard",
        "why_verify_identity",
        "wrong_amount_of_cash_received",
        "wrong_exchange_rate_for_cash_withdrawal"
    ],
    "dataset_schema": {
        "input_columns": [
            "example"
        ],
        "label_column": "label"
    },
    "seed_examples": [
        {
            "example": "Is it free to transfer money or is there a fee?",
            "label": "transfer_fee_charged"
        },
        {
            "example": "I need my PIN, where is it?",
            "label": "get_physical_card"
        },
        {
            "example": "Can my salary be received and transferred to my current currency in my country?",
            "label": "receiving_money"
        },
        {
            "example": "Why isn't my purchase exchange rate correct?",
            "label": "card_payment_wrong_exchange_rate"
        },
        {
            "example": "Why was I charged a fee on a cash withdrawal?",
            "label": "cash_withdrawal_charge"
        }
    ]
}

In [33]:
task_config = {
    "project_name": "BankingClassification",
    "task_type": "classification",
    "prefix_prompt": "You are an expert at understanding twitter complaints.",
    "chain_of_thought": "True"
}

In [34]:
o = LabelingAgent(task_config, llm_config)
o.plan('data/banking_test.csv', dataset_config, max_items = 100)

  0%|          | 0/5 [00:00<?, ?it/s]

2023-05-08 22:31:40.287 | INFO     | refuel_oracle.oracle:generate_explanations:423 - Chain of thought requires explanations for seed examples. Generating explanations for seed examples.


100%|██████████| 20/20 [00:00<00:00, 244.74it/s]

Total Estimated Cost: $0.179
Number of examples to label: 100
Average cost per example: $0.00179


A prompt example:

You are an expert at understanding twitter complaints.
Your job is to correctly label the provided input example into one of the following 77 categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_charge
exchange_rate
excha

In [35]:
labels, df, metrics_list = o.run('data/banking_test.csv', dataset_config, max_items = 100)

 25%|██▌       | 5/20 [01:24<04:07, 16.53s/it]

2023-05-08 22:33:56.520 | INFO     | refuel_oracle.tasks.base:parse_json_llm_response:131 - Error parsing LLM response: Let's think step by step.
This is not a complaint or issue related to any of the provided categories. It is a general question about the difference between two types of delivery options. Therefore, it does not fit into any of the categories and cannot be labeled.. AttributeError("'NoneType' object has no attribute 'strip'")


 80%|████████  | 16/20 [04:47<01:16, 19.11s/it]2023-05-08 22:37:33 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 289fb6d43caca91739ab3cbe2192a97f in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-08 22:37:33 /home/ubuntu/.local/lib/python3.8/site-packages/langchain/chat_models/openai.py WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 289fb6d43caca91739ab3cbe2192a97f in your message.).
100%|██████████| 20/20 [06:34<00:00, 19.71s

Metric: support: [(99, 'index=0')]
Metric: threshold: [(-inf, 'index=0')]
Metric: accuracy: [(0.696969696969697, 'index=0')]
Metric: completion_rate: [(0.99, 'index=0')]
Total number of failures: 0


## Notes

I am noting a few things that I did not try out, but would have wanted to try to boost completion rate while keeping the accuracy high

1. Wanted to try chain of thought reasoning with a semantic similarity example selector
2. Increase the number of seed examples to be close to one example per class
3. Find the annotator guidelines and pass these as part of the prompt to the model

Trying out all bells and whistles together

In [36]:
dataset_config = {
    "labels_list": [
        "activate_my_card",
        "age_limit",
        "apple_pay_or_google_pay",
        "atm_support",
        "automatic_top_up",
        "balance_not_updated_after_bank_transfer",
        "balance_not_updated_after_cheque_or_cash_deposit",
        "beneficiary_not_allowed",
        "cancel_transfer",
        "card_about_to_expire",
        "card_acceptance",
        "card_arrival",
        "card_delivery_estimate",
        "card_linking",
        "card_not_working",
        "card_payment_fee_charged",
        "card_payment_not_recognised",
        "card_payment_wrong_exchange_rate",
        "card_swallowed",
        "cash_withdrawal_charge",
        "cash_withdrawal_not_recognised",
        "change_pin",
        "compromised_card",
        "contactless_not_working",
        "country_support",
        "declined_card_payment",
        "declined_cash_withdrawal",
        "declined_transfer",
        "direct_debit_payment_not_recognised",
        "disposable_card_limits",
        "edit_personal_details",
        "exchange_charge",
        "exchange_rate",
        "exchange_via_app",
        "extra_charge_on_statement",
        "failed_transfer",
        "fiat_currency_support",
        "get_disposable_virtual_card",
        "get_physical_card",
        "getting_spare_card",
        "getting_virtual_card",
        "lost_or_stolen_card",
        "lost_or_stolen_phone",
        "order_physical_card",
        "passcode_forgotten",
        "pending_card_payment",
        "pending_cash_withdrawal",
        "pending_top_up",
        "pending_transfer",
        "pin_blocked",
        "receiving_money",
        "Refund_not_showing_up",
        "request_refund",
        "reverted_card_payment?",
        "supported_cards_and_currencies",
        "terminate_account",
        "top_up_by_bank_transfer_charge",
        "top_up_by_card_charge",
        "top_up_by_cash_or_cheque",
        "top_up_failed",
        "top_up_limits",
        "top_up_reverted",
        "topping_up_by_card",
        "transaction_charged_twice",
        "transfer_fee_charged",
        "transfer_into_account",
        "transfer_not_received_by_recipient",
        "transfer_timing",
        "unable_to_verify_identity",
        "verify_my_identity",
        "verify_source_of_funds",
        "verify_top_up",
        "virtual_card_not_working",
        "visa_or_mastercard",
        "why_verify_identity",
        "wrong_amount_of_cash_received",
        "wrong_exchange_rate_for_cash_withdrawal"
    ],
    "dataset_schema": {
        "input_columns": [
            "example"
        ],
        "label_column": "label"
    },
    "seed_examples": 'data/banking_seed.csv'
}

In [41]:
task_config = {
    "project_name": "BankingClassification",
    "task_type": "classification",
    "prefix_prompt": "You are an expert at understanding twitter complaints.",
    "example_selector": {
        "strategy": "semantic_similarity",
        "num_examples": 4
    },
    "compute_confidence": "True",
    "chain_of_thought": "True"
}

Increase the maximum tokens allowed on the llm in case of chain of thought because some times the number of tokens produced by the llm in the explanation would exceed the default max token limit.

In [44]:
llm_config = {
    "provider_name": "openai",
    "model_name": "gpt-3.5-turbo",
    "has_logprob": False,
    "model_params": {
        "max_tokens": 1000, # This increases the maximum tokens
    }
}

In [45]:
o = LabelingAgent(task_config, llm_config)
o.plan('data/banking_test.csv', dataset_config, max_items = 100)

100%|██████████| 20/20 [00:29<00:00,  1.49s/it]

Total Estimated Cost: $0.362
Number of examples to label: 100
Average cost per example: $0.00362


A prompt example:

You are an expert at understanding twitter complaints.
Your job is to correctly label the provided input example into one of the following 77 categories.
Categories:
activate_my_card
age_limit
apple_pay_or_google_pay
atm_support
automatic_top_up
balance_not_updated_after_bank_transfer
balance_not_updated_after_cheque_or_cash_deposit
beneficiary_not_allowed
cancel_transfer
card_about_to_expire
card_acceptance
card_arrival
card_delivery_estimate
card_linking
card_not_working
card_payment_fee_charged
card_payment_not_recognised
card_payment_wrong_exchange_rate
card_swallowed
cash_withdrawal_charge
cash_withdrawal_not_recognised
change_pin
compromised_card
contactless_not_working
country_support
declined_card_payment
declined_cash_withdrawal
declined_transfer
direct_debit_payment_not_recognised
disposable_card_limits
edit_personal_details
exchange_charge
exchange_rate
excha

In [46]:
labels, df, metrics_list = o.run('data/banking_test.csv', dataset_config, max_items = 100)

 20%|██        | 4/20 [01:34<06:23, 23.99s/it]2023-05-08 23:10:44 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 23c095ac766b84d7caf88df9a89b769a in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-05-08 23:10:44 /home/ubuntu/.local/lib/python3.8/site-packages/langchain/chat_models/openai.py WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 23c095ac766b84d7caf88df9a89b769a in your message.).
100%|██████████| 20/20 [08:09<00:00, 24.49s/

Metric: auroc: 0.8120978120978121
Metric: support: [(100, 'index=0'), (0, 'index=1'), (1, 'index=2'), (16, 'index=3'), (17, 'index=4'), (18, 'index=5'), (20, 'index=6'), (24, 'index=7'), (25, 'index=8'), (26, 'index=9'), (28, 'index=10'), (37, 'index=11'), (38, 'index=12'), (43, 'index=13'), (44, 'index=14'), (51, 'index=15'), (52, 'index=16'), (54, 'index=17'), (56, 'index=18'), (60, 'index=19'), (61, 'index=20'), (65, 'index=21'), (66, 'index=22'), (68, 'index=23'), (70, 'index=24'), (71, 'index=25'), (72, 'index=26'), (74, 'index=27'), (75, 'index=28'), (77, 'index=29'), (78, 'index=30'), (80, 'index=31'), (85, 'index=32'), (86, 'index=33'), (100, 'index=34')]
Metric: threshold: [(-inf, 'index=0'), (3.71441431254104, 'index=1'), (2.71441431254104, 'index=2'), (2.5839988367995232, 'index=3'), (2.5772682289443654, 'index=4'), (2.571900894553442, 'index=5'), (2.546606833621545, 'index=6'), (2.529675078736434, 'index=7'), (2.529153546390658, 'index=8'), (2.5165535885857393, 'index=9'), 

/home/ubuntu/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/ubuntu/.local/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Increase the maximum tokens allowed on the llm in case of chain of thought because some times the number of tokens produced by the llm in the explanation would exceed the default max token limit.

## Self consistency

This increases the temperature and the randomness while generating explanations allowing the model to explore multiple reasoning paths. At the end, a majority vote is taken among the generations. The llm config below generates 5 reasoning paths and takes the majority vote over these reasoning paths.

In [48]:
llm_config = {
    "provider_name": "openai",
    "model_name": "gpt-3.5-turbo",
    "has_logprob": False,
    "model_params": {
        "max_tokens": 1000, # This increases the maximum tokens
        "temperature": 0.7,
        "n": 5 # This runs self consistency
    }
}